## Code generation

In [1]:
# import sys
# !{sys.executable} -m pip install --upgrade pip
# !{sys.executable} -m pip install sympy
# !{sys.executable} -m pip install scipy
# !{sys.executable} -m pip install matplotlib

In [2]:
# Remove matplotlib deprecation warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [3]:
# initialize sympy printing (for latex output)
from sympy import init_printing, Symbol
init_printing()

# import functions and classes for compartment models
import sys
sys.path.insert(0, "./compartor") #use local copy
from compartor import *

Here, we illustrate how to export the moment equations in LaTeX format or generate code for simulations.

We consider again the exemplary model of the paper and derive moments equations with the automated function.

In [4]:
from sympy import symbols, log, exp, sin, cos, tan, sqrt

x = Content('x')
y = Content('y')

sumKernel = x[0] + y[0]
multKernel = x[0] * y[0]
freeMolecularKernel = (x[0]**(1/3) + y[0]**(1/3))**2 * sqrt( (1/x[0]) + (1/y[0]) )
diffusiveMolecularKernel = (x[0]**(-1/3) + y[0]**(-1/3)) * (x[0]**(1/3) + y[0]**(1/3))
gelationTestKernel = ( sqrt( (x[0]**2 * y[0])+(x[0] * y[0]**2) ) * ( x[0]**(1/3) + y[0]**(1/3) ) )

# Coagulation   = TransitionClass( [x] + [y] -to> [x+y],       'k_C',                  name='C')
Coagulation   = TransitionClass( [x] + [y] -to> [x+y],       'k_C', g=sumKernel,     name='C')
# Coagulation   = TransitionClass( [x] + [y] -to> [x+y],       'k_C', g=multKernel,     name='C')
# Coagulation   = TransitionClass( [x] + [y] -to> [x+y],       'k_C', g=freeMolecularKernel,     name='C')
# Coagulation   = TransitionClass( [x] + [y] -to> [x+y],       'k_C', g=gelationTestKernel,     name='C')

transitions = [Coagulation]
display_transition_classes(transitions)

[x] + [y] ---> [x + y],  h_C = k_C*(n(y) - KroneckerDelta(x, y))*(x[0] + y[0])*n(x)/(KroneckerDelta(x, y) + 1)

In [5]:
moments = [
    Moment(0),
    Moment(0)**2,
    Moment(1),
    Moment(1)**2,
    Moment(2),
]
display(moments)
equations = automated_moment_equations(1, transitions, moments, clna=True)
# equations = automated_moment_equations(1, transitions, moments, clna=False)
display_moment_equations(equations)

⎡                 2                      2              ⎤
⎣Moment(0), Moment (0), Moment(1), Moment (1), Moment(2)⎦

Computed moment equations for desired moments $\left< N\right> $, $\left< N^{2}\right> $, $\left< M^{1}\right> $, $\left< {\left(M^{1}\right)}^{\mathtt{\text{2}}}\right> $, and $\left< M^{2}\right> $.

Equations were iteratively added for $\left< N M^{1}\right> $.

d                                                                   
──(Expectation(N)) = k_C⋅(1 - Expectation(N))⋅Expectation(Moment(1))
dt                                                                  

d ⎛           ⎛ 2⎞⎞       ⎛             2                                                                                                                              ⎛ 2⎞   
──⎝Expectation⎝N ⎠⎠ = k_C⋅⎝2⋅Expectation (N)⋅Expectation(Moment(1)) - 2⋅Expectation(N)⋅Expectation(N⋅Moment(1)) + Expectation(N)⋅Expectation(Moment(1)) - 2⋅Expectation⎝N ⎠⋅Ex
dt                                                                                                                                                                            

                                                                          ⎞
pectation(Moment(1)) + 2⋅Expectation(N⋅Moment(1)) - Expectation(Moment(1))⎠
                                                                           

d                             
──(Expectation(Moment(1))) = 0
dt                            

d ⎛           ⎛      2   ⎞⎞    
──⎝Expectation⎝Moment (1)⎠⎠ = 0
dt                             

                                                                   3           
d                            2⋅k_C⋅(Expectation(N) - 1)⋅Expectation (Moment(1))
──(Expectation(Moment(2))) = ──────────────────────────────────────────────────
dt                                                       2                     
                                              Expectation (N)                  

d                                  ⎛                          2                                        ⎛      2   ⎞                                                           
──(Expectation(N⋅Moment(1))) = k_C⋅⎝Expectation(N)⋅Expectation (Moment(1)) - Expectation(N)⋅Expectation⎝Moment (1)⎠ - Expectation(N⋅Moment(1))⋅Expectation(Moment(1)) + Expect
dt                                                                                                                                                                            

     ⎛      2   ⎞⎞
ation⎝Moment (1)⎠⎠
                  

The LaTeX source of the ODE system can be found and copy-pasted by modifying the Math Render option of the jupyter notebook. This is done by right-clicking on the system and choosing Math Settings > Math Render > Plain Source .

From the closed equations, we can also generate code to simulate the system. Currently, Python or Julia code can be generated.

For a direct code output, the user can rely on the functions:

In [6]:
# python_code = generate_python_code(equations, function_name="coagulation_fragmentation")
# print(python_code)
# exec(python_code) 

In [7]:
julia_code = generate_julia_code(equations, function_name="IECFBDq")
print(julia_code)

# evaluate ODEs
function IECFBDq_ODEs(dM, M, parameters, t)
  c0 = parameters[:kC] # kC
  # Number of Compartments (N)
  dM[1] = c0*(1-M[1])*M[3]
  # N^2
  dM[2] = c0*(-M[3]+2*M[6]+M[1]*M[3]-2*M[2]*M[3]-2*M[6]*M[1]+2*((M[1])^(2))*M[3])
  # Total Mass
  dM[3] = 0
  # M^2
  dM[4] = 0
  # Sum of squared content
  dM[5] = 2*c0*((M[3])^(3))*(-1+M[1])/((M[1])^(2))
  # N*M
  dM[6] = c0*(((M[3])^(2))*M[1]-M[4]*M[1]-M[6]*M[3]+M[4])
  return
end

# initialize expected moments vector
function IECFBDq_initial(n0)
  M = zeros(6)
  # Number of Compartments (N)
  M[1] = ??? # initial value for Moment(0,), please specify!
  # N^2
  M[2] = ((M[1])^(2))
  # Total Mass
  M[3] = ??? # initial value for Moment(1,), please specify!
  # M^2
  M[4] = ((M[3])^(2))
  # Sum of squared content
  M[5] = ??? # initial value for Moment(2,), please specify!
  # N*M
  M[6] = M[1]*M[3]
  return M
end


